<a href="https://colab.research.google.com/github/Yk-rin04/Prog2kakushin/blob/main/work2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Work2
numpyを用いたプログラム

音の名前（A,B,C等），音の長さ，どのようにするか（連結または重ね合わせ）を入力することによって作曲ができるプログラム．

## 全体の機能  
曲作りのできるプログラムのところにある5つの異なるプログラムを自由に用いることで，作曲を行うことができる．このプログラムは一人の人が，途中でネット環境が途切れることなく，それほど大規模ではない曲を作ることを前提としてつくられている．  

# 準備

## 関数のインストール  
周期関数（例えば正弦波）のデータを音源に変換するために必要なライブラリをインストールする．

In [ ]:
import numpy as np
import IPython

## ライブラリの作成  
音の高さ，音の大きさ，音の長さに関するライブラリを作成  
・音の高さ  
音の名前に対してその音の周波数を返す辞書になっている．523.251Hzを中心として前後2オクターブ分の音に対応している．音の名前は周期的に繰り返されるため，523.251～987.767Hzより低い音は音の名前の頭に「l(lowの意味)」を，それより高い音は音の名前の頭に「h(highの意味)」をつけることで区別した．半音は#と♭の二種類があるが今回は#のみに対応しており，使う場合には音の名前の頭に「#」をつけることで機能するようにした．また，曲を作る人によって使いやすい音階名が異なると考えたため，ドレミ音階とドイツ音階の二つを用意した．  
・音の大きさ  
音の大きさを0～10で指定すると0.0～1.0に変更するという辞書を作成した．音量は小数で入力するという認識はあまりないと思ったため，今回はこのような辞書を作成した．  
・音の長さ  
四分音符の長さを1.00とした時の他の音の長さを数値化し，音の名前を入力することでその音の長さを返す辞書である．

In [ ]:
#音の高さ
TONES1 = {'R':0 ,'lC':261.626 ,'lC#':277.185 ,'lD':293.665 ,'lD#':311.127 ,'lE':329.628 ,'lF':349.228 ,'lF#':369.994 ,'lG':391.995 ,'lG#':415.305 ,'lA':440.000 ,'lA#':466.164 ,'lB':493.885 ,
         'C':523.251 ,'C#':554.365 ,'D':587.330 ,'D#':622.254 ,'E':659.255 ,'F':698.456 ,'F#':739.989 ,'G':783.991 ,'G#':830.609 ,'A':880.000 ,'A#':932.328 ,'B':987.767 ,'hC':1046.502}

TONES2 = {'R':0 ,'lド':261.626 ,'lド#':277.185 ,'lレ':293.665 ,'lレ#':311.127 ,'lミ':329.628 ,'lファ':349.228 ,'lファ#':369.994 ,'lソ':391.995 ,'lソ#':415.305 ,'lラ':440.000 ,'lラ#':466.164 ,'lシ':493.885 ,
         'ド':523.251 ,'ド#':554.365 ,'レ':587.330 ,'レ#':622.254 ,'ミ':659.255 ,'ファ':698.456 ,'ファ#':739.989 ,'ソ':783.991 ,'ソ#':830.609 ,'ラ':880.000 ,'ラ#':932.328 ,'シ':987.767 ,'hド':1046.502}

In [ ]:
#音の大きさ
BOLUME = {0:0.0, 1:0.1, 2:0.2, 3:0.3, 4:0.4, 5:0.5, 6:0.6, 7:0.7, 8:0.8, 9:0.9, 10:1.0}

In [ ]:
#音の長さ
LONG = {'16分音符':0.25, '8分音符':0.50, '4分音符':1.00, '2分音符':2.00, '符点2分音符':3.00, '全音符':4.00}

# 曲作りのできるプログラム  
5つのプログラムを使うことで，自分の作った曲を確認しながら保存したりして曲を作ることができる．  

プログラムの種類  
・1パート作成ができるプログラム  
・曲の保存ができるプログラム  
・曲の連結ができるプログラム  
・曲の合成ができるプログラム  
・保存した曲が聞けるプログラム

以下は使用する音階名や曲のテンポを決められるプログラム．  
同じ人が一つの曲を作成するという今回の前提においては，音階名や曲のテンポは一度決めてしまえばそのあと変更することはあまりないと思ったため，先に指定できるプログラムを作成した．

In [ ]:
# 音階名を入力
tone = input('ドイツ音階？ドレミ音階？:')
if tone == 'ドイツ音階':
  TONE = TONES1
elif tone == 'ドレミ音階':
  TONE = TONES2
else:
  print('どちらかから選んでください')

#テンポを入力
tempo = int(input('テンポを入力してください(半角数字で入力):'))
tem = 60 / tempo

ドイツ音階？ドレミ音階？:ドイツ音階
テンポを入力してください(半角数字で入力):60


##1パート作成できるプログラム  
音の名前，音の大きさ，音の長さを順次入力することで曲が作れるプログラムとなっている．最後に合計の拍数を表示することで，その後のプログラムの条件に使うための条件を満たしているか確認できるようになっている．

〈関数の説明と関数内でのルール〉  
pitch():音程を決める関数  
この関数は入力された音程に対する周波数を戻り値として返す  
・1オクターブ低い音はｌを，１オクターブ高い音はhを音の前につける  
・「#」を使う場合は音の後ろにつける (♭には対応していません)  
・休符はRを入力する
  

〈関数の説明と関数内でのルール〉  
pitch():音程を決める関数  
この関数は入力された音程に対する周波数を戻り値として返す  
・1オクターブ低い音はｌを，１オクターブ高い音はhを音の前につける  
・「#」を使う場合は音の後ろにつける (♭には対応していません)  
・休符はRを入力する  
・この条件が守られていない場合は守られるまで繰り返す  

bolume():音量を決める関数  
音量を半角数字で0～10で入力するとそれらを0.0～1.0に変換する関数  
・半角数字で入力しない場合や範囲外で入力した場合はエラーが発生してプログラムは止まる．  

long():音の長さを決める関数  
音の長さの名前を入力することでその音に対応した音の長さが返り値として指定される．  
・辞書LONGに存在しない音の長さの名前が入力された場合，もう一度入力させる仕組みになっている．

In [ ]:
#１パート作成
Fs = 48000
y1 = []
total = 0

def pitch():

  while True:
    pitch = input('音程を入力してください:')
    if pitch in TONE:
      break
    else:
      print('ルールを守ってください')
      continue
  tone = TONE[pitch]
  #print(tone)
  return tone


def bolume():
  bol = int(input('音量を0～10で入力してください:'))
  A = BOLUME[bol]
  #print(A)
  return A

def long():
  while True:
    long = input('音の長さの名前を入力(例：2分音符, 符点2分音符):')
    if long in LONG:
      break
    else:
      print('その長さは使用できません')
      continue
  d_t = LONG[long]
  #print(d_t)
  return d_t


while True:
  print()
  f = pitch()
  A = bolume()
  d_t = long()
  t = np.linspace(0., d_t, int(Fs*d_t*tem))
  y = A * np.sin(2.0*np.pi*f*t)
  y1 = np.append(y1, y)
  total += d_t
  
  choice2 = input('そのまま曲を続けますか　yes or no:')
  if choice2 == 'yes':
    continue
  else:
    break

print()
print('全体の曲の長さは{}拍です'.format(total))
IPython.display.Audio(y1, rate=Fs, autoplay=True, normalize=False)


音程を入力してください:G
音量を0～10で入力してください:3
音の長さの名前を入力(例：2分音符, 符点2分音符):8分音符
そのまま曲を続けますか　yes or no:yes

音程を入力してください:F
音量を0～10で入力してください:4
音の長さの名前を入力(例：2分音符, 符点2分音符):2分音符
そのまま曲を続けますか　yes or no:yes

音程を入力してください:A
音量を0～10で入力してください:6
音の長さの名前を入力(例：2分音符, 符点2分音符):8分音符
そのまま曲を続けますか　yes or no:yes

音程を入力してください:lC
音量を0～10で入力してください:5
音の長さの名前を入力(例：2分音符, 符点2分音符):8分音符
そのまま曲を続けますか　yes or no:no

全体の曲の長さは3.5拍です


## 曲の保存ができるプログラム  
他のプログラムを実行した後，保存したいと思ったときに利用できるプログラム．保存はgoogle colabを再起動するとリセットされる．
曲そのものを保存するファイルには後ろに.csv，曲の長さを保存するファイルには.totalをつけてそれぞれ保存される．  
なお，その後のファイル名の呼び出しの際には自分が入力したファイル名のみを入力すればよいものになっている.  
保存できるのは，保存したい曲を作った直後にしか行えない．

In [ ]:
#曲の保存
y2 = []

name = input('ファイルに名前を付けてください：')
np.savetxt('{}.csv'.format(name),y1,delimiter=',')

Total = np.append(y2,total)
np.savetxt('{}.total'.format(name),Total,delimiter=',')
print('保存が完了しました．')

ファイルに名前を付けてください：合成
保存が完了しました．


## 曲の連結ができるプログラム  
曲が保存された二つの.csvファイルの音源を連結させて新しい曲を作ることのできるプログラム．それぞれの音源の音の長さが保存された.totalファイルも同時に読み込み，連結させた曲の拍数も表示することができる．
また，このプログラムは連結したい二つの音源の長さが違っていても実行することができる．  

In [ ]:
#曲の連結
name1 = input('前半に連結したいファイル名を入力してください:')
name2 = input('後半に連結させたいファイル名を入力してください:')

data1 = np.loadtxt('{}.csv'.format(name1), delimiter=",", skiprows=0)
data2 = np.loadtxt('{}.csv'.format(name2), delimiter=",", skiprows=0)

total1 = np.loadtxt('{}.total'.format(name1), delimiter=",", skiprows=0)
total2 = np.loadtxt('{}.total'.format(name2), delimiter=",", skiprows=0)

y1 = np.append(data1, data2)

total = total1 + total2

print('全体の曲の長さは{}拍です'.format(total))
IPython.display.Audio(y1, rate=Fs, autoplay=True, normalize=False)

前半に連結したいファイル名を入力してください:CDE_3.5
後半に連結させたいファイル名を入力してください:GFAlC_3.5
全体の曲の長さは7.0拍です


## 曲の合成ができるプログラム  
曲が保存された二つの.csvファイルの音源を重ね，和音とした曲をつくれるプログラム．.totalファイルも同時に読み込み，もし二つのファイルの音源の長さが異なっていた場合には「曲の長さが違う」と表示する．また，二つの音源の長さが異なっていた場合にはプログラムはエラーを起こして停止する．

In [ ]:
#曲の合成
name1 = input('合成したいファイル名を入力してください:')
name2 = input('合成したいファイル名を入力してください:')

data1 = np.loadtxt('{}.csv'.format(name1), delimiter=",", skiprows=0)
data2 = np.loadtxt('{}.csv'.format(name2), delimiter=",", skiprows=0)

total1 = np.loadtxt('{}.total'.format(name1), delimiter=",", skiprows=0)
total2 = np.loadtxt('{}.total'.format(name2), delimiter=",", skiprows=0)

if total1 != total2:
  print('二つの音源の長さが異なります．')

total = total1

y = data1 + data2
max = np.max(y)
min = np.min(y)

if max > -min:
  MAX = max
else:
  MAX = min

y1 = y / MAX

print('全体の曲の長さは{}拍です'.format(total))

IPython.display.Audio(y1, rate=Fs, autoplay=True, normalize=False)

合成したいファイル名を入力してください:合成
合成したいファイル名を入力してください:7.0
二つの音源の長さが異なります．


ValueError: ignored

## 保存した曲が聞けるプログラム  
保存してある曲を再生できるプログラム．

In [ ]:
name = input('再生したいファイル名を入力してください:')
data = np.loadtxt('{}.csv'.format(name), delimiter=",", skiprows=0)
IPython.display.Audio(data, rate=Fs, autoplay=True, normalize=False)

再生したいファイル名を入力してください:CDE_3.5
